In [75]:
# 🧩 1. Import necessary libraries
import requests
import json
import time
import pandas as pd
import os

In [ ]:
# 🔐 2. Set API endpoints and keys
GRAPHQL_URL = "https://api.kpler.marinetraffic.com/v2/vessels/graphql"
GRAPHQL_API_KEY = ""
AIS_API_KEY = ""
HISTORICAL_API_KEY = ""
PORTCALLS_API_KEY = ""

In [77]:
# 📦 3. Fetch vessels by REGISTER_OWNER
def fetch_vessels(after_cursor=None):
    query = f"""
    query Vessels {{
        vessels(
            first: 1000
            where: {{
                filters: [
                    {{
                        field: "management.beneficialOwner.current.name"
                        op: LIKE
                        values: ["PETROLEOS DE VENEZUELA SA%"]
                    }}
                ]
                operator: OR
            }}
            after: {json.dumps(after_cursor)}
        ) {{
            nodes {{
                identifier {{
                    imo
                }}
            }}
            pageInfo {{
                hasNextPage
                endCursor
            }}
        }}
    }}
    """

    headers = {
        "Authorization": f"Basic {GRAPHQL_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(GRAPHQL_URL, json={"query": query}, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    return response.json()

In [78]:
# 📥 4. Loop through pages and gather IMO list
imo_list = []
after_cursor = None

while True:
    data = fetch_vessels(after_cursor)
    if not data:
        break

    vessels = data["data"]["vessels"]["nodes"]
    for vessel in vessels:
        imo = vessel["identifier"].get("imo")
        if imo:
            imo_list.append(imo)

    page_info = data["data"]["vessels"]["pageInfo"]
    if page_info["hasNextPage"]:
        after_cursor = page_info["endCursor"]
    else:
        break

print(f"✅ Found {len(imo_list)} vessels. First 5 IMOs: {imo_list[:5]}")

✅ Found 16 vessels. First 5 IMOs: [8022858, 8117512, 9038842, 9038866, 9117478]


In [79]:
# 📡 5. Fetch live AIS positions from MT Export API
def fetch_ais_data(api_key, imo_list, timespan=1440, buffer_time=1):
    url_template = f'https://services.marinetraffic.com/api/exportvessel/{api_key}/v:6/timespan:{timespan}/imo:{{imo}}/protocol:jsono'
    all_ais_data = []

    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching AIS for IMO: {imo}")
        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list):
                    for record in data:
                        record['IMO'] = imo
                        all_ais_data.append(record)
                else:
                    print(f"⚠️ Unexpected format for IMO {imo}")
            else:
                print(f"❌ Failed for IMO {imo}: {response.status_code}")
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        if idx < len(imo_list):
            time.sleep(buffer_time)

    if all_ais_data:
        df = pd.DataFrame(all_ais_data)
        if {'IMO', 'SHIPNAME', 'LAT', 'LON', 'TIMESTAMP'}.issubset(df.columns):
            return df[['IMO', 'SHIPNAME', 'LAT', 'LON', 'TIMESTAMP']]
        else:
            return df
    else:
        print("⚠️ No AIS data fetched.")
        return pd.DataFrame()

In [80]:
# 🔍 6. Call the function and show results
df_ais = fetch_ais_data(AIS_API_KEY, imo_list)
print(f"✅ Retrieved {len(df_ais)} AIS position records.")
df_ais.head()

[1/16] Fetching AIS for IMO: 8022858
[2/16] Fetching AIS for IMO: 8117512
[3/16] Fetching AIS for IMO: 9038842
[4/16] Fetching AIS for IMO: 9038866
[5/16] Fetching AIS for IMO: 9117478
[6/16] Fetching AIS for IMO: 9117492
[7/16] Fetching AIS for IMO: 9117507
[8/16] Fetching AIS for IMO: 9503586
[9/16] Fetching AIS for IMO: 9503598
[10/16] Fetching AIS for IMO: 9543500
[11/16] Fetching AIS for IMO: 9543512
[12/16] Fetching AIS for IMO: 9552496
[13/16] Fetching AIS for IMO: 9552501
[14/16] Fetching AIS for IMO: 9586722
[15/16] Fetching AIS for IMO: 9586734
[16/16] Fetching AIS for IMO: 9589748
✅ Retrieved 7 AIS position records.


,IMO,SHIPNAME,LAT,LON,TIMESTAMP
0,9038866,TESEO,11.746338,-70.216499,2025-04-22T10:49:48
1,9117507,NEGRA MATEA,38.472080,-8.793312,2025-04-22T10:48:11
2,9503586,ANITA,12.270220,-70.018936,2025-04-22T10:51:36
3,9543500,YARE,11.642587,-70.296906,2025-04-22T10:49:28
4,9543512,PARAMACONI,5.549700,0.087100,2025-04-22T06:19:46


In [81]:
# 💾 7. Save to CSV file
if not df_ais.empty:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    output_filename = f"ais_positions_pdvsa_{timestamp}.csv"
    df_ais.to_csv(output_filename, index=False)
    print(f"📁 Data saved to: {output_filename}")
else:
    print("⚠️ No data to save.")

📁 Data saved to: ais_positions_pdvsa_2025-04-22_18-54.csv


In [ ]:
# 📡 8. Fetch and append historical AIS per IMO
def fetch_historical_ais_and_save(api_key, imo_list, output_path, days=180, period="daily", msgtype="simple", buffer_time=60):
    url_template = f'https://services.marinetraffic.com/api/exportvesseltrack/{api_key}/v:3/days:{days}/period:{period}/imo:{{imo}}/msgtype:{msgtype}/protocol:jsono'

    # 🧾 Create CSV with headers if it doesn't exist
    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as f:
            pd.DataFrame(columns=['IMO', 'LAT', 'LON', 'TIMESTAMP']).to_csv(f, index=False)

    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching and saving AIS for IMO: {imo}")
        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list) and data:
                    for record in data:
                        record['IMO'] = imo
                    df = pd.DataFrame(data)
                    
                    # ✍️ Save only if essential columns exist
                    if {'IMO', 'LAT', 'LON', 'TIMESTAMP'}.issubset(df.columns):
                        df[['IMO', 'LAT', 'LON', 'TIMESTAMP']].to_csv(output_path, mode='a', header=False, index=False)
                        print(f"✅ Saved {len(df)} records for IMO {imo}")
                    else:
                        print(f"⚠️ Skipped saving due to missing fields for IMO {imo}")
                else:
                    print(f"⚠️ No data returned for IMO {imo}")
            else:
                print(f"❌ Failed request for IMO {imo}: {response.status_code}")
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        if idx < len(imo_list):
            time.sleep(buffer_time)


In [83]:
# 🔍 9. Call the historical AIS fetcher and preview
df_hist_ais = fetch_historical_ais(api_key=HISTORICAL_API_KEY, imo_list=imo_list, days=180, period="daily", buffer_time=60) # You can increase this if hitting rate limits

print(f"✅ Retrieved {len(df_hist_ais)} historical AIS position records.")

# 💾 10. Save to CSV
if not df_hist_ais.empty:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    output_filename = f"historical_ais_pdvsa_{timestamp}.csv"
    df_hist_ais.to_csv(output_filename, index=False)
    print(f"📁 Historical AIS data saved to: {output_filename}")
else:
    print("⚠️ No historical data to save.")


[1/16] Fetching historical AIS for IMO: 8022858
[2/16] Fetching historical AIS for IMO: 8117512
[3/16] Fetching historical AIS for IMO: 9038842
[4/16] Fetching historical AIS for IMO: 9038866
[5/16] Fetching historical AIS for IMO: 9117478
[6/16] Fetching historical AIS for IMO: 9117492
[7/16] Fetching historical AIS for IMO: 9117507
[8/16] Fetching historical AIS for IMO: 9503586
[9/16] Fetching historical AIS for IMO: 9503598
[10/16] Fetching historical AIS for IMO: 9543500
[11/16] Fetching historical AIS for IMO: 9543512
[12/16] Fetching historical AIS for IMO: 9552496
[13/16] Fetching historical AIS for IMO: 9552501
[14/16] Fetching historical AIS for IMO: 9586722
[15/16] Fetching historical AIS for IMO: 9586734
[16/16] Fetching historical AIS for IMO: 9589748
✅ Retrieved 1219 historical AIS position records.
📁 Historical AIS data saved to: historical_ais_pdvsa_2025-04-22_19-09.csv


In [87]:
# 📆 11. Derive dynamic date range from historical AIS DataFrame
def get_dynamic_date_range(df_hist_ais):
    if df_hist_ais.empty:
        raise ValueError("🚫 Historical AIS DataFrame is empty. Cannot derive dynamic date range.")

    df_hist_ais['TIMESTAMP'] = pd.to_datetime(df_hist_ais['TIMESTAMP'], errors='coerce')
    fromdate = df_hist_ais['TIMESTAMP'].min().strftime("%Y-%m-%d %H:%M")
    todate = df_hist_ais['TIMESTAMP'].max().strftime("%Y-%m-%d %H:%M")
    print(f"📆 Dynamic Date Range — From: {fromdate} | To: {todate}")
    return fromdate, todate

In [88]:
# 🧭 12. Fetch and append Port Calls using Port Calls API
def fetch_port_calls_and_save(
    api_key,
    imo_list,
    output_path,
    df_hist_ais,
    msgtype="simple",
    buffer_time=60
):
    # 🎯 Get dynamic fromdate and todate
    fromdate, todate = get_dynamic_date_range(df_hist_ais)

    # 🧾 Define required columns
    required_columns = ['IMO', 'SHIPNAME', 'TIMESTAMP_UTC', 'MOVE_TYPE', 'PORT_ID', 'PORT_NAME']
    
    # 📁 Ensure output CSV exists with headers
    if not os.path.exists(output_path):
        pd.DataFrame(columns=required_columns).to_csv(output_path, index=False)
        print(f"📄 Created new CSV file with headers: {output_path}")
    
    url_template = (
        f'https://services.marinetraffic.com/api/portcalls/{api_key}/v:6/'
        f'fromdate:{fromdate}/todate:{todate}/imo:{{imo}}/msgtype:{msgtype}/protocol:jsono'
    )

    # 🚀 Loop through each IMO
    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching Port Calls for IMO: {imo}")

        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list) and data:
                    # 🏷️ Tag records with IMO
                    for record in data:
                        record['IMO'] = imo
                    
                    df = pd.DataFrame(data)

                    # ✅ Save if valid
                    if set(required_columns).issubset(df.columns):
                        df[required_columns].to_csv(output_path, mode='a', header=False, index=False)
                        print(f"✅ Saved {len(df)} port call records for IMO {imo}")
                    else:
                        print(f"⚠️ Missing required fields — skipped saving for IMO {imo}")
                else:
                    print(f"⚠️ No port call data for IMO {imo}")
            else:
                print(f"❌ Failed request for IMO {imo}: {response.status_code}")
        
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        # ⏱️ Pause between calls to respect rate limits
        if idx < len(imo_list):
            time.sleep(buffer_time)


In [89]:
# 📁 13. Output file path
output_csv = f"port_calls_pdvsa_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"

# 📡 14. Run the fetcher with dynamic dates
fetch_port_calls_and_save(
    api_key=PORTCALLS_API_KEY,
    imo_list=imo_list,
    output_path=output_csv,
    df_hist_ais=df_hist_ais,  # Pass in the historical AIS dataframe
    buffer_time=60  # ⏳ Adjustable delay
)

📆 Dynamic Date Range — From: 2024-10-24 11:16 | To: 2025-04-22 06:01
📄 Created new CSV file with headers: port_calls_pdvsa_2025-04-22_19-16.csv
[1/16] Fetching Port Calls for IMO: 8022858
⚠️ No port call data for IMO 8022858
[2/16] Fetching Port Calls for IMO: 8117512
⚠️ No port call data for IMO 8117512
[3/16] Fetching Port Calls for IMO: 9038842
⚠️ No port call data for IMO 9038842
[4/16] Fetching Port Calls for IMO: 9038866
✅ Saved 24 port call records for IMO 9038866
[5/16] Fetching Port Calls for IMO: 9117478
⚠️ No port call data for IMO 9117478
[6/16] Fetching Port Calls for IMO: 9117492
✅ Saved 2 port call records for IMO 9117492
[7/16] Fetching Port Calls for IMO: 9117507
✅ Saved 16 port call records for IMO 9117507
[8/16] Fetching Port Calls for IMO: 9503586
✅ Saved 97 port call records for IMO 9503586
[9/16] Fetching Port Calls for IMO: 9503598
⚠️ No port call data for IMO 9503598
[10/16] Fetching Port Calls for IMO: 9543500
✅ Saved 2 port call records for IMO 9543500
[11/16]